In [28]:
import os
import numpy as np
import pandas as pd
from PIL import Image
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

In [27]:
images_dir = "training/training"
labels = pd.read_csv("training_labels.csv")

# add the directory to the filename
labels['ID'] = labels['ID'].apply(lambda x: os.path.join(images_dir, x))

# Initialize the ImageDataGenerator
# You can change the size of the validation split (0.25 is 25% of data used as validation set)
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.25)

# Create the training and validation generators
train_generator = datagen.flow_from_dataframe(
    dataframe=labels,
    directory=None,  
    x_col='ID',
    y_col='target',
    target_size=(224, 224), # You can change the size of the image
    batch_size=32, # You can change the batch_size
    class_mode='categorical',  
    subset='training'
)

validation_generator = datagen.flow_from_dataframe(
    dataframe=labels,
    directory=None,  
    x_col='ID',
    y_col='target',
    target_size=(224, 224), # Should match training size
    batch_size=32, # Should match training
    class_mode='categorical',  
    subset='validation'
)

Found 2591 validated image filenames belonging to 5 classes.
Found 863 validated image filenames belonging to 5 classes.


In [29]:
# Build the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(5, activation='softmax')  # Assuming 5 classes of flowers
])

In [30]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=10,  # You can adjust the number of epochs
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size
)

Epoch 1/10


80/80 [==============================] - 134s 2s/step - loss: 1.3766 - accuracy: 0.4193 - val_loss: 1.1699 - val_accuracy: 0.4700
Epoch 2/10
80/80 [==============================] - 91s 1s/step - loss: 1.0529 - accuracy: 0.5729 - val_loss: 1.0697 - val_accuracy: 0.5877
Epoch 3/10
80/80 [==============================] - 90s 1s/step - loss: 0.8587 - accuracy: 0.6702 - val_loss: 1.1198 - val_accuracy: 0.5769
Epoch 4/10
80/80 [==============================] - 90s 1s/step - loss: 0.6662 - accuracy: 0.7405 - val_loss: 1.0068 - val_accuracy: 0.6310
Epoch 5/10
80/80 [==============================] - 90s 1s/step - loss: 0.4283 - accuracy: 0.8496 - val_loss: 1.1556 - val_accuracy: 0.6262
Epoch 6/10
80/80 [==============================] - 90s 1s/step - loss: 0.2511 - accuracy: 0.9175 - val_loss: 1.3829 - val_accuracy: 0.6274
Epoch 7/10
80/80 [==============================] - 90s 1s/step - loss: 0.1386 - accuracy: 0.9590 - val_loss: 1.6923 - val_accuracy: 0.5817
Epoch 8/10
80/80 

In [ ]:
# Load the test data
test_images_dir = "testing/"
test_image_paths = [os.path.join(test_images_dir, f) for f in os.listdir(test_images_dir) if f.endswith(('.png', '.jpg', '.jpeg'))]

In [ ]:
# Preprocess test images (resize and rescale)
test_images = np.array([np.array(Image.open(img).resize((224, 224))) / 255. for img in test_image_paths])

In [ ]:
# Generate predictions for the test set
test_predictions = model.predict(test_images)

In [ ]:
# Get class labels from generator
class_labels = list(train_generator.class_indices.keys())

In [ ]:
# Convert predictions to class labels
predicted_labels = [class_labels[np.argmax(pred)] for pred in test_predictions]

In [ ]:
# Generate IDs for the test set
test_ids = [os.path.splitext(os.path.basename(path))[0] for path in test_image_paths]

In [ ]:
# Create a DataFrame for submission
submission_df = pd.DataFrame({'ID': test_ids, 'Prediction': predicted_labels})